Notes:
On Fidelity, the annual Opening and Closing prices are calculated as follows:\
Opening: The price found in the open column of january\
Closing: The price found in the close column of december

In [1]:
import pandas as pd
import os

In [2]:
#convert datetime column to an int year
def convert_date_to_year(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].dt.year
    return df

In [3]:
#convert Open, High, Low, Close, % Change, % Change vs Average, Volume to numeric. Adjust for commas
def convert_numeric(df, col):
    #print(col)
    if df[col].dtype == 'object':
        df[col] = pd.to_numeric(df[col].str.replace(',', ''), errors='coerce')
    return df

In [4]:
#convert all the columns
def convert_all_numeric(df):
    df = convert_numeric(df, 'Open')
    df = convert_numeric(df, 'High')
    df = convert_numeric(df, 'Low')
    df = convert_numeric(df, 'Close')
    df = convert_numeric(df, 'Volume')
    return df

In [5]:
#based on closing prices of each year, find the rolling CAGR of each year, for each amount of time
def calculate_cagr(df):
    df['5YRCAGR'] = ((df['Close'] / df['Close'].shift(5)) ** (1/5) - 1) * 100
    df['10YRCAGR'] = ((df['Close'] / df['Close'].shift(10)) ** (1/10) - 1) * 100
    df['15YRCAGR'] = ((df['Close'] / df['Close'].shift(15)) ** (1/15) - 1) * 100
    df['20YRCAGR'] = ((df['Close'] / df['Close'].shift(20)) ** (1/20) - 1) * 100
    df['25YRCAGR'] = ((df['Close'] / df['Close'].shift(25)) ** (1/25) - 1) * 100
    df['30YRCAGR'] = ((df['Close'] / df['Close'].shift(30)) ** (1/30) - 1) * 100
    return df

In [6]:
#round the CAGR columns to 3 decimal places
def round_cagr(df):
    df['5YRCAGR'] = df['5YRCAGR'].round(3)
    df['10YRCAGR'] = df['10YRCAGR'].round(3)
    df['15YRCAGR'] = df['15YRCAGR'].round(3)
    df['20YRCAGR'] = df['20YRCAGR'].round(3)
    df['25YRCAGR'] = df['25YRCAGR'].round(3)
    df['30YRCAGR'] = df['30YRCAGR'].round(3)
    return df

In [7]:

for filename in os.listdir('Historical_Data'):
    if filename.endswith(".csv"):
        #read historical stock market data
        df = pd.read_csv('Historical_Data/' + filename)
        
        #cleaning up columns by renaming them and converting to proper data types
        df = convert_date_to_year(df)
        df = convert_all_numeric(df)
        df = df.rename(columns={'% Change': '1YRCAGR'})
        
        #calculate the compound annual growth rates
        df = calculate_cagr(df)
        df = round_cagr(df)
        
        #keep only the year and CAGR columns, and save to CAGR directory
        df = df[['Year', 'Open','Close','1YRCAGR', '5YRCAGR', '10YRCAGR', '15YRCAGR', '20YRCAGR', '25YRCAGR', '30YRCAGR']]
        df.to_csv('CAGR/' + filename, index=False)